In [1]:
import pandas as pd

In [3]:
from nycflights13 import flights
from nycflights13 import airlines
from nycflights13 import airports

In [10]:
flights.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z


In [11]:
airports.head()

,faa,name,lat,lon,alt,tz,dst,tzone
0,04G,Lansdowne Airport,41.130472,-80.619583,1044,-5,A,America/New_York
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264,-6,A,America/Chicago
2,06C,Schaumburg Regional,41.989341,-88.101243,801,-6,A,America/Chicago
3,06N,Randall Airport,41.431912,-74.391561,523,-5,A,America/New_York
4,09J,Jekyll Island Airport,31.074472,-81.427778,11,-5,A,America/New_York


In [12]:
airlines.head()

,carrier,name
0,9E,Endeavor Air Inc.
1,AA,American Airlines Inc.
2,AS,Alaska Airlines Inc.
3,B6,JetBlue Airways
4,DL,Delta Air Lines Inc.


In [15]:
delayed_flights = flights[flights['dep_delay'] > 0].groupby('carrier').size().reset_index(name="num_delay")
delayed_flights

,carrier,num_delay
0,9E,7063
1,AA,10162
2,AS,226
3,B6,21445
4,DL,15241
5,EV,23139
6,F9,341
7,FL,1654
8,HA,69
9,MQ,8031


In [16]:
delayed_flights.merge(airlines,on="carrier",how="left")

,carrier,num_delay,name
0,9E,7063,Endeavor Air Inc.
1,AA,10162,American Airlines Inc.
2,AS,226,Alaska Airlines Inc.
3,B6,21445,JetBlue Airways
4,DL,15241,Delta Air Lines Inc.
5,EV,23139,ExpressJet Airlines Inc.
6,F9,341,Frontier Airlines Inc.
7,FL,1654,AirTran Airways Corporation
8,HA,69,Hawaiian Airlines Inc.
9,MQ,8031,Envoy Air


In [17]:
flights = flights[flights['dep_delay'] > 0] # filter for delays
carrier_delay_count = flights.groupby('carrier').size().reset_index(name='num_delay') # count the delays by carrier
max_delay_carrier = carrier_delay_count[carrier_delay_count['num_delay'] == carrier_delay_count['num_delay'].max()] # find the carrier with the most delays
most_delayed_airline = pd.merge(max_delay_carrier, airlines, on='carrier', how='left')['name'] # join with airlines table and select the name of the most delayed airline

In [18]:
most_delayed_airline

0    United Air Lines Inc.
Name: name, dtype: object

In [20]:
flights.groupby('dest').agg(mean_delay=('arr_delay','mean'))

,mean_delay
dest,
ABQ,26.460870
ACK,31.950617
ALB,44.000000
ANC,0.166667
ATL,41.692537
...,...
TPA,34.570677
TUL,64.970414
TVC,62.028571


In [21]:
flights.groupby('dest').agg(mean_delay=('arr_delay'
                                        , 'mean')).reset_index().query('mean_delay == mean_delay.min()')

,dest,mean_delay
75,PSP,-9.333333


In [22]:
flights.groupby('dest').agg(mean_delay=('arr_delay', 'mean')).reset_index()\
    .query('mean_delay == mean_delay.min()').merge(airports, left_on='dest', right_on='faa', how='left')

,dest,mean_delay,faa,name,lat,lon,alt,tz,dst,tzone
0,PSP,-9.333333,PSP,Palm Springs Intl,33.829667,-116.506694,477,-8,A,America/Los_Angeles
